Para ejecutar los siguientes cuadernos, si aún no lo has hecho, necesitas establecer la clave de OpenAI dentro del archivo .env como `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

A continuación, vamos a cargar el Índice de Embeddings en un DataFrame de Pandas. El Índice de Embeddings está almacenado en un archivo JSON llamado `embedding_index_3m.json`. El Índice de Embeddings contiene los Embeddings de cada una de las transcripciones de YouTube hasta finales de octubre de 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

A continuación, vamos a crear una función llamada `get_videos` que buscará en el Índice de Embeddings la consulta proporcionada. La función devolverá los 5 videos que sean más similares a la consulta. El funcionamiento de la función es el siguiente:

1. Primero, se crea una copia del Índice de Embeddings.
2. Luego, se calcula el Embedding para la consulta utilizando la API de Embeddings de OpenAI.
3. Después, se crea una nueva columna en el Índice de Embeddings llamada `similarity`. Esta columna contiene la similitud coseno entre el Embedding de la consulta y el Embedding de cada segmento de video.
4. A continuación, se filtra el Índice de Embeddings por la columna `similarity`. Solo se incluyen los videos que tengan una similitud coseno mayor o igual a 0.75.
5. Finalmente, el Índice de Embeddings se ordena por la columna `similarity` y se devuelven los 5 videos principales.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Esta función es muy simple, solo imprime los resultados de la consulta de búsqueda.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Primero, el Índice de Embedding se carga en un Dataframe de Pandas.
2. Después, se le pide al usuario que ingrese una consulta.
3. Luego, se llama a la función `get_videos` para buscar la consulta en el Índice de Embedding.
4. Finalmente, se llama a la función `display_results` para mostrar los resultados al usuario.
5. Después, se le pide al usuario que ingrese otra consulta. Este proceso continúa hasta que el usuario escriba `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.es.png)

Se te pedirá que ingreses una consulta. Escribe una consulta y presiona enter. La aplicación te mostrará una lista de videos que son relevantes para la consulta. Además, la aplicación te dará un enlace al momento exacto del video donde se encuentra la respuesta a la pregunta.

Aquí tienes algunas consultas para probar:

- ¿Qué es Azure Machine Learning?
- ¿Cómo funcionan las redes neuronales convolucionales?
- ¿Qué es una red neuronal?
- ¿Puedo usar Jupyter Notebooks con Azure Machine Learning?
- ¿Qué es ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o inexactitudes. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que surjan del uso de esta traducción.
